In [1]:
import numpy as np
import pandas as pd
from itertools import product
from collections import Counter
from datasets import load_dataset
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

In [2]:
def metric_calculation(pred, gt):    
    acc=accuracy_score(gt, pred)
    f1=f1_score(gt, pred, average='macro')
    confusion=confusion_matrix(gt, pred)
    fpr=confusion[0,1]/len(gt) ## predict to be 1; actual 0
    fnr=confusion[1,0]/len(gt) ## predict to be 0; actual 1
    return acc, f1, fpr, fnr

In [3]:
def post_processing(pred):
    new_pred=[]
    for i in pred:
        i=i.lower()
        if 'response' in i:
            try: new_pred.append(i.split('response')[1].split()[1].replace('</s>', ''))
            except: new_pred.append(2)
        elif 'output' in i:
            try: new_pred.append(i.split('output')[1].split()[1].replace('</s>', ''))
            except: new_pred.append(2)
        else:
            try: new_pred.append(i.split()[0].replace('</s>', ''))
            except:new_pred.append(2)
    new_pred = np.array([int(float(i)) if i in ['0', '0.0', '1', '1.0'] else 2 for i in new_pred])
    return new_pred

In [4]:
ds = load_dataset("beanham/spatial_join_dataset")
test=ds['test']
gt=np.array(test['label'])

In [5]:
models = ['llama3', '4o_mini', 'gemini', 'qwen']
methods = ['zero_shot', 'few_shot'] 
modes = ['no_heur', 'with_heur_hint', 'with_heur_value']
heuristics = ['angle', 'distance', 'comb']
configs=['_'.join(i) for i in list(product(methods, modes, heuristics))]
configs.remove('zero_shot_no_heur_angle')
configs.remove('zero_shot_no_heur_distance')
configs.remove('zero_shot_no_heur_comb')
configs.remove('few_shot_no_heur_angle')
configs.remove('few_shot_no_heur_distance')
configs.remove('few_shot_no_heur_comb')
configs.insert(0, 'zero_shot_no_heur')
configs.insert(7, 'few_shot_no_heur')

In [22]:
## evaluate on a subset
np.random.seed(100)
index=np.random.randint(0, len(test), 1000)

In [23]:
index[:10]

array([1544, 2840, 1859,   79, 1930,  350, 1972, 1890,   53, 1602])

In [8]:
## heuristcis
for i in [1,2,5,10,20]:
    pred=np.array(test['min_angle'])<=i
    metrics=metric_calculation(pred[index], gt[index])
    print(i, metrics[0], metrics[1])

1 0.867 0.8649569334461061
2 0.908 0.9047760996290408
5 0.946 0.9429281655844156
10 0.948 0.9445921985815603
20 0.934 0.9288927961938254


In [9]:
results=[]
for model in models:
    print(f'Model: {model}...')
    for config in configs:
        pred=np.load(f'base/{model}/{model}_{config}.npy')
        pred=post_processing(pred)        
        metrics=metric_calculation(pred[index], gt[index])
        results.append([config, model, metrics[0], metrics[1]])
results=pd.DataFrame(results, columns=['config', 'model', 'acc', 'f1'])

Model: llama3...
Model: 4o_mini...
Model: gemini...
Model: qwen...


In [10]:
results[results.model=='qwen']

,config,model,acc,f1
42,zero_shot_no_heur,qwen,0.603,0.392110
43,zero_shot_with_heur_hint_angle,qwen,0.645,0.496562
44,zero_shot_with_heur_hint_distance,qwen,0.609,0.405572
45,zero_shot_with_heur_hint_comb,qwen,0.676,0.564495
46,zero_shot_with_heur_value_angle,qwen,0.946,0.942522
47,zero_shot_with_heur_value_distance,qwen,0.732,0.656840
48,zero_shot_with_heur_value_comb,qwen,0.950,0.946998
49,few_shot_no_heur,qwen,0.633,0.565963
50,few_shot_with_heur_hint_angle,qwen,0.682,0.657786
51,few_shot_with_heur_hint_distance,qwen,0.679,0.655147
